In [32]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import difflib
import sys

stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

def bbb(a,b):
    c = {}
    for i,j in eval(a).items():
        c[i.lower()] = j

    if b.lower() in c:
        if float(c[b.lower()])>0.1:
            return 1
        else:
            return 0
    else:
        return 0

def nunique(data_df):
    # add nunique feature
    label_feature=['title', 'tag', 'prefix']
    data_temp = data_df[label_feature]
    df_feature = pd.DataFrame()
    data_temp['cnt']=1
    print('Begin stat...')
    col_type = label_feature[:]
    n = len(col_type)
    for i in range(n):
        for j in range(n):
            if i!=j:
                col_name = "count_type_"+col_type[j]+"_in_"+col_type[i]
                print(col_name)
                se = data_temp.groupby([col_type[i]])[col_type[j]].value_counts()
                se = pd.Series(1,index=se.index).sum(level=col_type[i])
                df_feature[col_name] = (data_temp[col_type[i]].map(se)).fillna(value=0).astype(int).values
    data_df = pd.concat([data_df, df_feature], axis=1)
    print('The end')
    return data_df

def extract_proba(pred):
    pred = eval(pred)
    pred = sorted(pred.items(), key=lambda x: x[1], reverse=True)
    pred_proba_lst=[]
    for i in range(10):
        if len(pred)<i+2:
            pred_proba_lst.append(0)
        else:
            pred_proba_lst.append(float(pred[i][1]))
    return pred_proba_lst

def extract_prefix_pred_similarity(lst):
    pred = eval(lst[1])
    pred = sorted(pred.items(), key=lambda x: x[1], reverse=True)
    prefix_pred_sim=[]
    for i in range(10):
        if len(pred)<i+2:
            prefix_pred_sim.append(0)
        else:
            prefix_pred_sim.append(difflib_similarity(lst[0],pred[i][0]))
    return prefix_pred_sim

def difflib_similarity(str1,str2):
    return difflib.SequenceMatcher(a=str1, b=str2).quick_ratio()

def extract_title_pred_similarity(lst):
    pred = eval(lst[1])
    pred = sorted(pred.items(), key=lambda x: x[1], reverse=True)
    title_pred_sim=[]
    for i in range(10):
        if len(pred)<i+2:
            title_pred_sim.append(0)
        else:
            title_pred_sim.append(difflib_similarity(lst[0],pred[i][0]))
    return title_pred_sim

train_data = pd.read_table('data/oppo_round1_train_20180929.txt', 
        names= ['prefix','query_prediction','title','tag','label'], header= None, encoding='utf-8').astype(str)
val_data = pd.read_table('data/oppo_round1_vali_20180929.txt', 
        names = ['prefix','query_prediction','title','tag','label'], header = None, encoding='utf-8').astype(str)
test_data = pd.read_table('data/oppo_round1_test_B_20181106.txt',
        names = ['prefix','query_prediction','title','tag'],header = None, encoding='utf-8').astype(str)
train_data = train_data[train_data['label'] != '音乐' ]
test_data['label'] = -1

train_data = pd.concat([train_data,val_data])
train_data['label'] = train_data['label'].apply(lambda x: int(x))
test_data['label'] = test_data['label'].apply(lambda x: int(x))
items = ['prefix', 'title', 'tag']

train_data['prefix_num'] = train_data['prefix'].apply(lambda x:len(x))
train_data['title_num'] = train_data['title'].apply(lambda x:len(x))
val_data['prefix_num'] = val_data['prefix'].apply(lambda x:len(x))
val_data['title_num'] = val_data['title'].apply(lambda x:len(x))
test_data['prefix_num'] = test_data['prefix'].apply(lambda x:len(x))
test_data['title_num'] = test_data['title'].apply(lambda x:len(x))

test_data['in_query_big'] = test_data.apply(lambda x:bbb(x['query_prediction'],x['title']),axis=1)


temp1 = train_data.groupby(['prefix','title','tag'],as_index=False)['label'].agg({'click1':'sum','count1':'count','ctr1':'mean'})
temp2 = train_data.groupby(['prefix','title'],as_index=False)['label'].agg({'click2':'sum','count2':'count','ctr2':'mean'})
temp3 = train_data.groupby(['prefix','tag'],as_index=False)['label'].agg({'click3':'sum','count3':'count','ctr3':'mean'})
temp4 = train_data.groupby(['title','tag'],as_index=False)['label'].agg({'click4':'sum','count4':'count','ctr4':'mean'})
temp5 = train_data.groupby('prefix',as_index=False)['label'].agg({'click5':'sum','count5':'count','ctr5':'mean'})
temp6 = train_data.groupby('title',as_index=False)['label'].agg({'click6':'sum','count6':'count','ctr6':'mean'})
temp7 = train_data.groupby('tag',as_index=False)['label'].agg({'click7':'sum','count7':'count','ctr7':'mean'})
temp8 = train_data.groupby(['prefix_num','title_num','tag'],as_index=False)['label'].agg({'click8':'sum','count8':'count','ctr8':'mean'})

train_data = pd.merge(train_data,temp1,on=['prefix','title','tag'],how='left')
train_data = pd.merge(train_data,temp2,on=['prefix','title'],how='left')
train_data = pd.merge(train_data,temp3,on=['prefix','tag'],how='left')
train_data = pd.merge(train_data,temp4,on=['title','tag'],how='left')
train_data = pd.merge(train_data,temp5,on='prefix',how='left')
train_data = pd.merge(train_data,temp6,on='title',how='left')
train_data = pd.merge(train_data,temp7,on='tag',how='left')
train_data = pd.merge(train_data,temp8,on=['prefix_num','title_num','tag'],how='left')

train_data = nunique(train_data)

print('pred proba starting')
train_data['pred_proba_lst']=train_data['query_prediction'].apply(extract_proba)
print('prefix pred starting')
train_data['prefix_pred_sim']=train_data[['prefix','query_prediction']].apply(extract_prefix_pred_similarity,axis=1)
print('title pred starting')
train_data['title_pred_sim']=train_data[['title','query_prediction']].apply(extract_title_pred_similarity,axis=1)

print('-----------add max similarity-----------')
print('---最大统计概率对应的prefix和title的相似度---')
train_data['pred_proba_lst_max'] = train_data['pred_proba_lst'].apply(lambda x: max(x))
train_data['prefix_pred_sim_max'] = train_data['prefix_pred_sim'].apply(lambda x: max(x)) 
train_data['title_pred_sim_max'] = train_data['title_pred_sim'].apply(lambda x: max(x))
train_data['pred_proba_lst_std'] = train_data['pred_proba_lst'].apply(lambda x: np.std(x))
train_data['prefix_pred_sim_std'] = train_data['prefix_pred_sim'].apply(lambda x: np.std(x)) 
train_data['title_pred_sim_std'] = train_data['title_pred_sim'].apply(lambda x: np.std(x))
train_data['proba_max_prefix'] = train_data['prefix_pred_sim'].apply(lambda x: x[0])
train_data['proba_max_title'] = train_data['title_pred_sim'].apply(lambda x: x[0])
    
print('-----------add mean similarity----------')
def do_mean(li):
    sums = 0
    for i in li:
        sums = sums + i
    return sums/len(li)
train_data['pred_proba_lst_mean'] = train_data['pred_proba_lst'].apply(do_mean)
train_data['prefix_pred_sim_mean'] = train_data['prefix_pred_sim'].apply(do_mean)
train_data['title_pred_sim_mean'] = train_data['title_pred_sim'].apply(do_mean)

print('-------add prefix and title similar-----')
def prefix_title_len_ratio(lst):
    return lst[0]/lst[1]
def prefix_title_sim(lst):
    return difflib_similarity(lst[0],lst[1])
train_data['prefix_title_len_ratio']=train_data[['prefix_len','title_len']].apply(prefix_title_len_ratio,axis=1)
train_data['prefix_title_sim']=train_data[['prefix','title']].apply(prefix_title_sim,axis=1)

temp = train_data.drop_duplicates(subset=['prefix','query_prediction','title','tag'], keep='first')
test_data = pd.merge(test_data,temp,on=['prefix', 'query_prediction', 'title', 'tag'], how='left')

for item in items:
    temp = train_data.groupby(item, as_index = False)['label'].agg({item+'_click':'sum', item+'_count':'count'})
    temp[item+'_ctr'] = temp[item+'_click']/(temp[item+'_count'])
    train_data = pd.merge(train_data, temp, on=item, how='left')
    test_data = pd.merge(test_data, temp, on=item, how='left')
for i in range(len(items)):
    for j in range(i+1, len(items)):
        item_g = [items[i], items[j]]
        temp = train_data.groupby(item_g, as_index=False)['label'].agg({'_'.join(item_g)+'_click': 'sum','_'.join(item_g)+'count':'count'})
        temp['_'.join(item_g)+'_ctr'] = temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'count']+3)
        train_data = pd.merge(train_data, temp, on=item_g, how='left')
        test_data = pd.merge(test_data, temp, on=item_g, how='left')
train_data_ = train_data.drop(['prefix', 'query_prediction', 'title', 'tag'], axis = 1)
test_data_ = test_data.drop(['prefix', 'query_prediction', 'title', 'tag'], axis = 1)

print('train beginning')

X = np.array(train_data_.drop(['label'], axis = 1))
y = np.array(train_data_['label'])
X_test_ = np.array(test_data_.drop(['label'], axis = 1))
print('================================')
print(X.shape)
print(y.shape)
print('================================')


xx_logloss = []
xx_submit = []
N = 5
skf = StratifiedKFold(n_splits=N, random_state=42, shuffle=True)

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 32,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

for k, (train_in, test_in) in enumerate(skf.split(X, y)):
    print('train _K_ flod', k)
    X_train, X_test, y_train, y_test = X[train_in], X[test_in], y[train_in], y[test_in]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=5000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=50,
                    verbose_eval=50,
                    )
    print(f1_score(y_test, np.where(gbm.predict(X_test, num_iteration=gbm.best_iteration)>0.5, 1,0)))
    xx_logloss.append(gbm.best_score['valid_0']['binary_logloss'])
    xx_submit.append(gbm.predict(X_test_, num_iteration=gbm.best_iteration))

print('train_logloss:', np.mean(xx_logloss))
s = 0
for i in xx_submit:
    s = s + i

test_data_['label'] = list(s / N)
test_data_['label'] = test_data_['label'].apply(lambda x: round(x))
print('test_logloss:', np.mean(test_data_.label))
# test_data_['label'].to_csv('./submit/result.csv',index = False)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Begin stat...
count_type_tag_in_title
count_type_prefix_in_title
count_type_title_in_tag
count_type_prefix_in_tag
count_type_title_in_prefix
count_type_tag_in_prefix
The end
pred proba starting
prefix pred starting
title pred starting
-----------add max similarity-----------
---最大统计概率对应的prefix和title的相似度---
-----------add mean similarity----------
-------add prefix and title similar-----


KeyError: "['prefix_len' 'title_len'] not in index"

In [8]:
xx_submit

[array([9.99302977e-01, 9.99528460e-01, 9.99546078e-01, ...,
        2.63483729e-04, 9.94167279e-01, 2.68654578e-04]),
 array([9.99291229e-01, 9.99519363e-01, 9.99529645e-01, ...,
        2.63285567e-04, 9.94545834e-01, 2.69333972e-04]),
 array([9.99399175e-01, 9.99470192e-01, 9.99470695e-01, ...,
        3.07095537e-04, 9.93640365e-01, 3.13136706e-04]),
 array([9.99336470e-01, 9.99531192e-01, 9.99549158e-01, ...,
        2.63306641e-04, 9.94218023e-01, 2.69523119e-04]),
 array([9.99419109e-01, 9.99477110e-01, 9.99485258e-01, ...,
        2.92052061e-04, 9.93824843e-01, 2.96584788e-04])]

In [9]:
s

array([4.99674896e+00, 4.99752632e+00, 4.99758083e+00, ...,
       1.38922354e-03, 4.97039634e+00, 1.41723316e-03])

In [22]:
test_data_['label'] = list(s / N)
test_data_['label'] = test_data_['label'].apply(lambda x: np.where(x>0.9995, 1,0))
print('test_logloss:', np.mean(test_data_.label))

('test_logloss:', 0.3475)


In [13]:
list(s / N)

[0.9993497918046564,
 0.9995052632552521,
 0.9995161667552859,
 0.9943724543672916,
 0.9994923289324713,
 0.9991025667676731,
 0.9995141673277068,
 0.9994103936685461,
 0.999493538249604,
 0.9993904922685939,
 0.9948986005656444,
 0.9995161667552859,
 0.9950976208460831,
 0.9940792688239724,
 0.9994854276338137,
 0.9994625054891102,
 0.9995117932525828,
 0.9994706210992236,
 0.9995161667552859,
 0.9995036907766901,
 0.9995123520480702,
 0.9994475020620921,
 0.9995117932525828,
 0.9994198248178889,
 0.9995370962402148,
 0.9994446935309774,
 0.0002799006444161943,
 0.9995133463778348,
 0.0002778447072438085,
 0.9994122627884563,
 0.9993514114303175,
 0.9995161667552859,
 0.9995144709841399,
 0.9994118603695066,
 0.9995133258086633,
 0.9993583707918713,
 0.9994849701209896,
 0.9995278399388754,
 0.9995025408945576,
 0.9995239631505115,
 0.9995074231036984,
 0.9995065415613478,
 0.9995370962402148,
 0.9992354494709156,
 0.9995044204011986,
 0.9994625054891102,
 0.9995004851177214,
 0.99410

In [23]:
train_data_.columns

Index([u'label', u'prefix_num', u'title_num', u'count1', u'ctr1', u'click1',
       u'ctr2', u'ctr3', u'ctr4', u'ctr5', u'ctr6', u'ctr7', u'ctr8',
       u'count2', u'count3', u'count4', u'count5', u'count6', u'count7',
       u'prefix_count', u'prefix_click', u'prefix_ctr', u'title_click',
       u'title_count', u'title_ctr', u'tag_click', u'tag_count', u'tag_ctr',
       u'prefix_title_click', u'prefix_titlecount', u'prefix_title_ctr',
       u'prefix_tag_click', u'prefix_tagcount', u'prefix_tag_ctr',
       u'title_tag_click', u'title_tagcount', u'title_tag_ctr'],
      dtype='object')

In [27]:
def nunique(data_df):
    # add nunique feature
    label_feature=['title', 'tag', 'prefix']
    data_temp = data_df[label_feature]
    df_feature = pd.DataFrame()
    data_temp['cnt']=1
    print('Begin stat...')
    col_type = label_feature[:]
    n = len(col_type)
    for i in range(n):
        for j in range(n):
            if i!=j:
                col_name = "count_type_"+col_type[j]+"_in_"+col_type[i]
                print(col_name)
                se = data_temp.groupby([col_type[i]])[col_type[j]].value_counts()
                se = pd.Series(1,index=se.index).sum(level=col_type[i])
                df_feature[col_name] = (data_temp[col_type[i]].map(se)).fillna(value=0).astype(int).values
    data_df = pd.concat([data_df, df_feature], axis=1)
    print('The end')
    return data_df

nunique(test_data)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Begin stat...
count_type_tag_in_title
count_type_prefix_in_title
count_type_title_in_tag
count_type_prefix_in_tag
count_type_title_in_prefix
count_type_tag_in_prefix
The end


,prefix,query_prediction,title,tag,label,prefix_num,title_num,in_query_big,count1,ctr1,...,prefix_tag_ctr,title_tag_click,title_tagcount,title_tag_ctr,count_type_tag_in_title,count_type_prefix_in_title,count_type_title_in_tag,count_type_prefix_in_tag,count_type_title_in_prefix,count_type_tag_in_prefix
0,蒲公英,"{""蒲公英根"": ""0.013"", ""蒲公英茶可以天天喝吗"": ""0.013"", ""蒲公英茶...",蒲公英,应用,-1,9,9,0,73.0,0.013699,...,0.013158,1.0,73.0,0.013158,2,2,1557,2833,4,5
1,中国邮政,"{""中国邮政快递"": ""0.020"", ""中国邮政网络学院"": ""0.015"", ""中国邮政...",中国邮政储蓄银行官网,网站,-1,12,30,0,147.0,0.585034,...,0.355482,131.0,242.0,0.534694,1,3,665,998,3,2
2,38度,"{""38度6黑龙在线听"": ""0.014"", ""38度6歌词"": ""0.026"", ""38度...",38度6 (DJ小鱼儿版),音乐,-1,5,23,0,23.0,0.608696,...,0.531915,18.0,29.0,0.562500,1,1,1426,1470,1,1
3,香辣酱,"{""香辣酱批发厂家"": ""0.012"", ""香辣酱怎么吃"": ""0.011"", ""香辣酱制作...",自制牛肉香辣酱的做法,菜谱,-1,9,30,0,3.0,0.000000,...,0.200000,0.0,3.0,0.000000,1,1,1440,1462,2,1
4,周迅,"{""周迅高圣远"": ""0.010"", ""周迅年龄"": ""0.028"", ""周迅个人资料"": ...",周迅吧,百科,-1,6,9,0,127.0,0.062992,...,0.505976,8.0,127.0,0.061538,1,1,11658,11739,2,2
5,最强,"{""最强军婚首长求轻宠"": ""0.017"", ""最强神话帝皇"": ""0.016"", ""最强升...",最强警少,阅读,-1,6,12,0,134.0,0.014925,...,0.014493,2.0,134.0,0.014599,1,1,1531,1527,3,3
6,众,"{""众泰t300"": ""0.024"", ""众泰t700"": ""0.036"", ""众泰t600...",众泰,汽车,-1,3,6,0,80.0,0.862500,...,0.457317,69.0,80.0,0.831325,1,2,303,327,2,2
7,作,"{""作业帮免费下载"": ""0.025"", ""作业帮"": ""0.141"", ""作文"": ""0....",学而思网校,应用,-1,3,15,0,1385.0,0.012996,...,0.372819,45.0,2299.0,0.019548,1,13,1557,2833,4,3
8,微博,"{""微博下载"": ""0.005"", ""微博登录网页版"": ""0.006"", ""微博名字大全""...",微博,应用,-1,6,6,0,748.0,0.497326,...,0.264609,433.0,1162.0,0.371674,1,8,1557,2833,3,2
9,汽车,"{""汽车报价"": ""0.010"", ""汽车标志大全"": ""0.020"", ""汽车之家"": ""...",汽车之家,快应用,-1,6,12,1,6170.0,0.036629,...,0.036611,1520.0,10689.0,0.142162,4,6,82,132,2,3


In [ ]:
"""
获取prefix,title与query_prediction中每个预测词的相似度,获取统计概率列表
"""
def extract_proba(pred):
    pred = eval(pred)
    pred = sorted(pred.items(), key=lambda x: x[1], reverse=True)
    pred_proba_lst=[]
    for i in range(10):
        if len(pred)<i+2:
            pred_proba_lst.append(0)
        else:
            pred_proba_lst.append(float(pred[i][1]))
    return pred_proba_lst

def extract_prefix_pred_similarity(lst):
    pred = eval(lst[1])
    pred = sorted(pred.items(), key=lambda x: x[1], reverse=True)
    prefix_pred_sim=[]
    for i in range(10):
        if len(pred)<i+2:
            prefix_pred_sim.append(0)
        else:
            prefix_pred_sim.append(difflib_similarity(lst[0],pred[i][0]))
    return prefix_pred_sim

def difflib_similarity(str1,str2):
    return difflib.SequenceMatcher(a=str1, b=str2).quick_ratio()

def extract_title_pred_similarity(lst):
    pred = eval(lst[1])
    pred = sorted(pred.items(), key=lambda x: x[1], reverse=True)
    title_pred_sim=[]
    for i in range(10):
        if len(pred)<i+2:
            title_pred_sim.append(0)
        else:
            title_pred_sim.append(difflib_similarity(lst[0],pred[i][0]))
    return title_pred_sim

print('pred proba starting')
data['pred_proba_lst']=data['query_prediction'].apply(extract_proba)
print('prefix pred starting')
data['prefix_pred_sim']=data[['prefix','query_prediction']].apply(extract_prefix_pred_similarity,axis=1)
print('title pred starting')
data['title_pred_sim']=data[['title','query_prediction']].apply(extract_title_pred_similarity,axis=1)

In [ ]:
print('-----------add max similarity-----------')
print('---最大统计概率对应的prefix和title的相似度---')
data['pred_proba_lst_max'] = data['pred_proba_lst'].apply(lambda x: max(x))
data['prefix_pred_sim_max'] = data['prefix_pred_sim'].apply(lambda x: max(x)) 
data['title_pred_sim_max'] = data['title_pred_sim'].apply(lambda x: max(x))
data['pred_proba_lst_std'] = data['pred_proba_lst'].apply(lambda x: np.std(x))
data['prefix_pred_sim_std'] = data['prefix_pred_sim'].apply(lambda x: np.std(x)) 
data['title_pred_sim_std'] = data['title_pred_sim'].apply(lambda x: np.std(x))
data['proba_max_prefix'] = data['prefix_pred_sim'].apply(lambda x: x[0])
data['proba_max_title'] = data['title_pred_sim'].apply(lambda x: x[0])
    
print('-----------add mean similarity----------')
def do_mean(li):
    sums = 0
    for i in li:
        sums = sums + i
    return sums/len(li)
data['pred_proba_lst_mean'] = data['pred_proba_lst'].apply(do_mean)
data['prefix_pred_sim_mean'] = data['prefix_pred_sim'].apply(do_mean)
data['title_pred_sim_mean'] = data['title_pred_sim'].apply(do_mean)

print('-------add prefix and title similar-----')
def prefix_title_len_ratio(lst):
    return lst[0]/lst[1]
def prefix_title_sim(lst):
    return difflib_similarity(lst[0],lst[1])
data['prefix_title_len_ratio']=data[['prefix_len','title_len']].apply(prefix_title_len_ratio,axis=1)
data['prefix_title_sim']=data[['prefix','title']].apply(prefix_title_sim,axis=1)

In [ ]:
# 由于很多样本相同，所以去重后，构造特征，再合并
data = data_df.drop_duplicates(subset=['prefix','query_prediction','title','tag'], keep='first')

In [30]:
import difflib